In [1]:
# This strategy works on support and resistance. Buying on support and selling on resistance.

In [2]:
!pip install websocket-client

     |████████████████████████████████| 71kB 2.5MB/s 


In [3]:
import websocket
import numpy as np
import requests
import json


In [4]:
symbol_1 = 'ADAUSDT'
symbol_2 = 'adausdt'
interval = '1m'

root_url = 'https://api.binance.com/api/v1/klines'
url = root_url + '?symbol=' + symbol_1 + '&interval=' + interval
data = json.loads(requests.get(url).text)


In [5]:
data = data[-30:]

In [6]:
socket = f'wss://stream.binance.com:9443/ws/{symbol_2}@kline_{interval}'

In [7]:
amount_left = 10
# avg_26, avg_9 = 0, 0
amt_coin = 0
portfolio = 0


local_lowest = 1000000000000
local_highest = 0
global_lowest = 1000000000000
global_highest = 0

opens, highs, lows, closes  = [], [], [], []

In [8]:
for i in data:
  opens.append(i[1])
  highs.append(i[2])
  lows.append(i[3])
  closes.append(i[4])

In [9]:
opens = [float(i) for i in opens] 
highs = [float(i) for i in highs] 
lows = [float(i) for i in lows] 
closes = [float(i) for i in closes] 

In [13]:
# Buying and Selling functions

def buy(allocated_money, price):
  global portfolio, amount_left
  amt_coin = allocated_money/price

def sell(allocated_money, price):
  global portfolio, amount_left
  amt_coin = 0


In [16]:
def on_message(ws,message):
    global portfolio, investment, closes, highs, lows, money_end, core_to_trade, core_quantity, real_time_portfolio_value
    global local_highest, local_lowest, amount_left
    json_message = json.loads(message)
    cs = json_message['k']
    candle_closed, open, close, high, low = cs['x'], cs['o'], cs['c'], cs['h'], cs['l']
    print(candle_closed)

    if candle_closed:
        opens.append(float(open))
        closes.append(float(close))
        highs.append(float(high))
        lows.append(float(low))
        last_price = closes[-1]
        print(f'Closes: {closes}')
        if(last_price < local_lowest):
            local_lowest = last_price
        if(last_price > local_highest):
            local_highest = last_price
        print(f'Lowest_Point: {local_lowest}')
        print(f'Highest_Point: {local_highest}')
        
        
        if(last_price < closes[-6]):
            local_lowest = last_price
        if(last_price > closes[-6]):
            local_highest = last_price


        portfolio = amt_coin*last_price

        # If last price < 95% of Max price then sell
        if ((amount_left!=0) and (last_price < local_highest * 0.993)):
            sell(-amount_left, price=last_price)
            print(f'We sold ${-amount_left} worth of bitcoin', '\n', '\n') 
            portfolio =  amount_left
            amt_coin = amount_left/last_price
            amount_left = 0
             

        # If last price > 105% of Min price then buy   
        if ((amount_left == 0) and (last_price > local_lowest * 1.007)):
            buy(amount_left, price=last_price)
            print(f'We bought ${portfolio} worth of bitcoin', '\n', '\n')
            amount_left = portfolio
            portfolio = 0
            amt_coin = 0
            
            
        print(money_end)
        print((local_highest-local_lowest)/last_price*100)
        print("\n")

#     if trade_amt > 0:
#       buy(trade_amt, price=last_price)
#       print(f'We bought ${trade_amt} worth of bitcoin', '\n', '\n')
#     elif trade_amt < 0:
#       sell(-trade_amt, price=last_price)
#       print(f'We sold ${-trade_amt} worth of bitcoin', '\n', '\n')
    
    
ws = websocket.WebSocketApp(socket, on_message=on_message)

In [17]:
ws.run_forever()

Streaming output truncated to the last 5000 lines.
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Closes: [1.8574, 1.8579, 1.854, 1.8528, 1.8525, 1.8562, 1.8632, 1.8573, 1.8604, 1.859, 1.857, 1.8564, 1.8564, 1.8552, 1.8554, 1.8574, 1.8566, 1.8613, 1.8605, 1.863, 1.8648, 1.8636, 1.8678, 1.8656, 1.8639, 1.8642, 1.864, 1.8649, 1.8644, 1.8615, 1.823, 1.8237, 1.8256, 1.8243, 1.8277, 1.8291, 1.8259, 1.8247, 1.8222, 1.8234, 1.8196, 1.8174, 1.8217, 1.8243, 1.8244, 1.8193, 1.8201, 1.818, 1.8186, 1.8157, 1.8126, 1.8123, 1.8157, 1.8165, 1.8176, 1.8211, 1.8201, 1.8214, 1.82, 1.8219, 1.821, 1.8136, 1.8124, 1.8121, 1.8121, 1.8152, 1.8118, 1.8161, 1.8165, 1.8169, 1.818, 1.8163, 1.8199, 1.8191, 1.8202, 1.823, 1.8244, 1.8242, 1.8252, 1.8224, 1.8202, 1.8202, 1.8259, 1.8236, 1.8275, 1.8289]
Lowest_Point: 1.8236
Highest_Point: 1.8289
False
False
False
False
False
False
False
False
False
False
False
False
False


False